[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/nvidia-llm-colab/blob/main/nemo_2b_bf16_tp1.ipynb)

In [ ]:
%cd /content
!apt -y install -qq aria2
!npm install -g localtunnel
!git clone https://huggingface.co/camenduru/apex
# !git clone https://github.com/NVIDIA/apex.git
%cd /content/apex
# !git checkout 03c9d80ed54c0eaa5b581bf42ceca3162f085327
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" --global-option="--fast_layer_norm" --global-option="--distributed_adam" --global-option="--deprecated_fused_adam" ./
!pip install nemo_toolkit['nlp']==1.17.0

%cd /content
!git clone https://github.com/NVIDIA/NeMo.git 
%cd /content/NeMo/examples/nlp/language_modeling
!git checkout v1.17.0
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/nvidia/GPT-2B-001/resolve/main/GPT-2B-001_bf16_tp1.nemo -d /content/NeMo/examples/nlp/language_modeling -o nemo_2b_bf16_tp1.nemo

import subprocess
import threading
import time
import socket
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread(target=iframe_thread, daemon=True, args=(5555,)).start()

!python megatron_gpt_eval.py trainer.precision=bf16 gpt_model_file=nemo_2b_bf16_tp1.nemo server=True tensor_model_parallel_size=1 trainer.devices=1